# IEMS 450-2 Programming Assignment 2 
## (Active-Set Method for Inequality-Constrained QPs)

Created: March 15, 2023

Author: Jonathan Bosnich

Purpose: Programming Assignment #2 for Nonlinear Optimization (IEMS 450-2)

Description: <br>
This program implements the active-set method for solving inequality-constrained QPs. <br>
For solving the EQP subproblem, I solve the KKT system directly. <br>
This method is then applied to numerical experiments: <br>
    - two given inequality-constrained QPs from the book <br>
    - randomly generated inequality-constrained QPs <br>

In [435]:
# Import numpy for numerical computations
import numpy as np

### Initialize options for the active-set algorithm

In [436]:
def InitOptions():
    """
    Initialize algorithm options with default values.

    Return
        options:
            This is a dictionary with fields that correpond to algorithmic
            options for the active-set method: In particular,
            
            max_iter:
                Maximum number of iterations
            tol:
                Convergence tolerance
            output_level:
                Amount of output printed
                0: No output
                1: Only summary information
                2: One line per iteration
    """
    # Create options dictionary with default values
    options = {}
    options["max_iter"] = 1e4
    options["tol"] = 1e-6
    options["output_level"] = 2

    return options

### EQP Solver

In [437]:
def solveEQP_Direct(W_k, G, c, A, b):
    """
    Description:
    This function solve the EQP subproblem in the active-set method.
    It directly solves the KKT system to get an optimal solution for
    the optimal primal and dual variables of the EQP.

    Input arguments:
        W_k: (m, 1) boolean array, current working set
        G: (n, n) array, defines the quadratic terms in objective
        c: (n, 1) array, defines linear terms in objective
        A: (m, n) array, defines linear terms in constraints
        b: (m, 1) array, defines constant terms in constraints

    Return values:
        xEQP: (n, 1) array, primal solution of the EQP
        lamEQP: (m, 1) array, corresponding dual solution (multipliers)
    """

    # First, trim A and b given the current working set
    inactive = [i for i in range(len(b)) if not W_k[i]]  # Indices of constraints not in W_k
    A_k = np.delete(A, inactive, 0)
    b_k = np.delete(b, inactive, 0)

    # If A_k is empty, solve the unconstrained system
    if len(A_k) == 0:
        xEQP = np.linalg.solve(G, -c)
        lamEQP = []
    else:
        # Construct the KKT system
        m, n = np.shape(A_k)

        K_U = np.concatenate((G, A_k.T), axis=1)
        K_L = np.concatenate((A_k, np.zeros([m, m])), axis=1)
        K = np.concatenate((K_U, K_L), axis=0)

        K_vec = np.concatenate((c, -b_k), axis=0)  # Right hand side vector of KKT system

        # Solve the linear system
        # Note: np.linalg.solve uses an LU decomposition
        # making it relatively computationally efficient
        K_sol = np.linalg.solve(K, K_vec)
        xEQP = -K_sol[:n]
        lamEQP = K_sol[n:]

    return xEQP, lamEQP

### Active-Set Method: Main Loop 

In [438]:
def solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options):
    """
    Description:
    This is the implementation of active-set method for solving IQPs.
    The gist is that you begin with an working set, which defines the
    inequality constraints that hold with equality.

    Thus, this defines an EQP which we can solve. Given the solution
    of the EQP, we either update the primal solution or the working
    set (through the algorithmic steps below), until the primal 
    solution equals the EQP solution and the dual vairables are
    all non-negative. 

    Input arguments:
        x_0: (n, 1) array, initial starting point
        W_0: (m, 1) boolean array, itinial working set
        G: (n, n) array, defines the quadratic terms in objective
        c: (n, 1) array, defines linear terms in objective
        A: (m, n) array, defines linear terms in constraints
        b: (m, 1) array, defines constant terms in constraints
        options:
            This is a dictionary with options for the algorithm.
            For details see the InitOptions() function.

    Return values:
        x_sol: (n, 1) array, optimal primal solution
        lam_sol: (m, 1) array, optimal dual solution
        W_sol: (m, 1) boolean array, final working set at optimal solution
        status:
            Return code indicating reason for termination:
            0:  Optimal solution found (convergence tolerance satisfied 
                and all multipliers are non-negative)
            -1:  Maximum number of iterations exceeded in the active-set loop
            -99:  Unknown error (bug?)
        stats:
            Return conditions that caused termination and overall runtime
            num_iter: number of total iterations
            norm_pk: infinity norm of final step size (should be < tol)
            min_lam: smallest value of multiplier (should be >= -tol)
    """

    # Initialize iteration counter
    iter = 0

    # Initialize return flag (set to -99 so that we do not accidentally
    # declare success)
    status = -99

    # Initialize local variables
    x_k = np.copy(x_0)
    W_k = np.copy(W_0)

    # Initial objective value and size of working set
    obj_val = 0.5*x_k.T@G@x_k + x_k.T@c
    W_size = len([i for i in W_k if i])
    W_newIndex = 0
    a_k = 0
    norm_pk = 0

    # Print header and zero-th iteration for output
    if options["output_level"] >= 2:
        output_header = '%6s %23s %9s %9s %11s %9s' % \
            ('iter', 'f', '||p_k||', 'alpha', 'new_index', 'size(W)')
        print(output_header)
        print('%6i %23.16e %9.2e %9.2e %11i %9i' %
            (iter, obj_val, norm_pk, a_k, W_newIndex, W_size))
    
    # Begin loop
    while 1:

        # First, if we've exceeded max number of iterations
        if iter >= options['max_iter']:
            # Set flag to indicate the maximum number of iterations has been exceeded
            status = -1
            break

        # Now, solve the EQP subproblem
        xEQP, lamEQP = solveEQP_Direct(W_k, G, c, A, b)

        # Compute step direction
        p_k = xEQP - x_k

        # Initialize that no change will be made to working set
        W_newIndex = 0

        # Check if p meets termination condition
        if np.linalg.norm(p_k, np.inf) < options["tol"]:  # that is, x ~ xEQP
            
            # Now, we look at the Lagrange multiplies
            all_pos = True
            for lam in lamEQP:
                if not lam >= -options["tol"]:
                    all_pos = False
                    break
            
            # If all multipliers are non-negative, we're done!
            if all_pos:
                # Second termination condition has been met
                status = 0
                break

            else:
                most_neg = min(lamEQP)  # Determine most negative multiplier
                
                # Convert to list and use .index() which returns first iterate by default
                i_remove = lamEQP.tolist().index(most_neg)
                
                # Update working set
                active_ind = [i for i in range(len(W_k)) if W_k[i]]  # Indices of active constraints
                W_k[active_ind[i_remove]] = False  # Remove specified constraint
                W_newIndex = -(active_ind[i_remove] + 1)  # Offset indexing at zero
    
                # note: keep x_k the same, i.e. x_{k+1} = x_k
        else:
            # Compute step size
            blocking_indices = []  # indices when blocking constraint conditions are met
            blocking_values = []
            for i, active in enumerate(W_k):
                    if not active:
                        ai = A[i, :]
                        if ai.T@p_k < 0:
                            blocking_indices.append(i)
                            blocking_values.append((b[i] - ai.T@x_k)/(ai.T@p_k))

            # Now, from Eq. (16.41) in the book, alpha is given by
            if len(blocking_values) == 0:
                a_k = 1
            else:
                a_k = min(1, min(blocking_values))
            x_k = x_k + a_k*p_k  # Update solution

            # Did we bump into a blocking constraint?
            if a_k < 1:
                i_add = blocking_indices[0]  # Choose first constraint to add
                W_k[i_add] = True  # Update working set
                W_newIndex = i_add + 1  # offset Python's indexing beginning at zero
        
        iter += 1  # Update iteration
        obj_val = 0.5*x_k.T@G@x_k + x_k.T@c  # New function value
        W_size = len([i for i in W_k if i])  # New size of working set
        norm_pk = np.linalg.norm(p_k, np.inf)  # New norm of p_k

        # Iteration output
        if options["output_level"] >= 2:
            # Print the output header every 10 iterations
            if iter % 10 == 0:
                print(output_header)
            print('%6i %23.16e %9.2e %9.2e %11i %9i' %
                (iter, obj_val, norm_pk, a_k, W_newIndex, W_size))
        
    # Finalize results
    x_sol = x_k
    lam_sol = lamEQP
    W_sol = W_k
    W_constraints = [i + 1 for i in range(len(W_sol)) if W_sol[i]]  # specificy constraints, not bools

    # Set the statistics
    # These are statistics relavant to the termination conditions
    # and overall runtime of the algorithm
    stats = {}
    stats['num_iter'] = iter
    stats['norm_pk'] = norm_pk
    stats['min_lam'] = min(lam_sol)
    stats['W_size'] = W_size

    # Final output message
    if options["output_level"] >= 1:
        print('Final primal solution.........: ', x_sol[:10])
        print('Final dual solution.........: ', lam_sol[:10])
        print('Final working set.........: ', W_constraints[:10])
        print('Final objective.................: %g' % obj_val)
        print('Number of iterations............: %d' % iter)
        if status == 0:
            print('Exit: Solution found!')
        elif status == -1:
            print('Exit: Maximum number of iterations (%d) exceeded.' %
                  iter)
        else:
            print('ERROR: Unknown status value: %d\n' % status)

    return x_sol, lam_sol, W_sol, status, stats


# Numerical Experiments

### (A) Solve Example 16.4 with $x_0 = (2,0)^T$ and $\mathcal{W}_0 = \emptyset$

In [439]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])
b = np.array([-2, -6, -2, 0, 0])

# Initial iterate and working set
x_0 = np.array([2, 0])
W_0 = [False for i in range(len(b))]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -6.4444444444444446e+00  2.50e+00  6.67e-01           1         1
     2 -6.4500000000000011e+00  6.67e-02  1.00e+00           0         1
Final primal solution.........:  [1.4 1.7]
Final dual solution.........:  [0.8]
Final working set.........:  [1]
Final objective.................: -6.45
Number of iterations............: 2
Exit: Solution found!


### (B) Solve Example 16.4 with $x_0 = (2,0)^T$ and $\mathcal{W}_0 = \{3, 5\}$

In [440]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])
b = np.array([-2, -6, -2, 0, 0])

# Initial iterate and working set
x_0 = np.array([2, 0])
W_0 = [False for i in range(len(b))]
W_0[2] = True
W_0[4] = True

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         2
     1  0.0000000000000000e+00  8.88e-16  0.00e+00          -3         1
     2 -1.0000000000000000e+00  1.00e+00  1.00e+00           0         1
     3 -1.0000000000000000e+00  0.00e+00  1.00e+00          -5         0
     4 -6.2500000000000000e+00  2.50e+00  6.00e-01           1         1
     5 -6.4500000000000011e+00  4.00e-01  1.00e+00           0         1
Final primal solution.........:  [1.4 1.7]
Final dual solution.........:  [0.8]
Final working set.........:  [1]
Final objective.................: -6.45
Number of iterations............: 5
Exit: Solution found!


### (C) Solve Example 16.4 with $x_0 = (2,0)^T$ and $\mathcal{W}_0 = \{3 \}$

In [441]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])
b = np.array([-2, -6, -2, 0, 0])

# Initial iterate and working set
x_0 = np.array([2, 0])
W_0 = [False for i in range(len(b))]
W_0[2] = True

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         1
     1 -4.9999999999999822e-02  2.00e-01  1.00e+00           0         1
     2 -4.9999999999999822e-02  0.00e+00  1.00e+00          -3         0
     3 -6.4500000000000011e+00  2.40e+00  6.67e-01           1         1
Final primal solution.........:  [1.4 1.7]
Final dual solution.........:  [0.8]
Final working set.........:  [1]
Final objective.................: -6.45
Number of iterations............: 3
Exit: Solution found!


### (D) Solve Example 16.4 with $x_0 = (2,0)^T$ and $\mathcal{W}_0 = \{5 \}$

In [442]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-2, -5])
A = np.array([
    [1, -2],
    [-1, -2],
    [-1, 2],
    [1, 0],
    [0, 1]
])
b = np.array([-2, -6, -2, 0, 0])

# Initial iterate and working set
x_0 = np.array([2, 0])
W_0 = [False for i in range(len(b))]
W_0[4] = True

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         1
     1 -1.0000000000000000e+00  1.00e+00  1.00e+00           0         1
     2 -1.0000000000000000e+00  0.00e+00  1.00e+00          -5         0
     3 -6.2500000000000000e+00  2.50e+00  6.00e-01           1         1
     4 -6.4500000000000011e+00  4.00e-01  1.00e+00           0         1
Final primal solution.........:  [1.4 1.7]
Final dual solution.........:  [0.8]
Final working set.........:  [1]
Final objective.................: -6.45
Number of iterations............: 4
Exit: Solution found!


### (E) Solve Problem (16.76) with $x_0 = (0,0)^T$ and $\mathcal{W}_0 = \emptyset$

In [443]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-6, -4])
A = np.array([
    [-1, -1],
    [1, 0],
    [0, 1]
])
b = np.array([-3, 0, 0])

# Initial iterate and working set
x_0 = np.array([0, 0])
W_0 = [False for i in range(len(b))]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -1.0919999999999998e+01  3.00e+00  6.00e-01           1         1
     2 -1.1000000000000000e+01  2.00e-01  1.00e+00           0         1
Final primal solution.........:  [2. 1.]
Final dual solution.........:  [2.]
Final working set.........:  [1]
Final objective.................: -11
Number of iterations............: 2
Exit: Solution found!


### (F) Solve Problem (16.76) with $x_0 = (0,0)^T$ and $\mathcal{W}_0 = \{2,3 \}$

In [444]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-6, -4])
A = np.array([
    [-1, -1],
    [1, 0],
    [0, 1]
])
b = np.array([-3, 0, 0])

# Initial iterate and working set
x_0 = np.array([0, 0])
W_0 = [False for i in range(len(b))]
W_0[1] = True
W_0[2] = True

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         2
     1  0.0000000000000000e+00  0.00e+00  0.00e+00          -2         1
     2 -9.0000000000000000e+00  3.00e+00  1.00e+00           0         1
     3 -9.0000000000000000e+00  0.00e+00  1.00e+00          -3         0
     4 -9.0000000000000000e+00  2.00e+00 -0.00e+00           1         1
     5 -1.1000000000000000e+01  1.00e+00  1.00e+00           0         1
Final primal solution.........:  [2. 1.]
Final dual solution.........:  [2.]
Final working set.........:  [1]
Final objective.................: -11
Number of iterations............: 5
Exit: Solution found!


### (G) Solve Problem (16.76) with $x_0 = (1,1)^T$ and $\mathcal{W}_0 = \emptyset$

In [445]:
options = InitOptions()  # Import options

# Define constrained problem
G = np.array([[2, 0], [0, 2]])
c = np.array([-6, -4])
A = np.array([
    [-1, -1],
    [1, 0],
    [0, 1]
])
b = np.array([-3, 0, 0])

# Initial iterate and working set
x_0 = np.array([1, 1])
W_0 = [False for i in range(len(b))]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0 -8.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -1.0777777777777777e+01  2.00e+00  3.33e-01           1         1
     2 -1.1000000000000000e+01  3.33e-01  1.00e+00           0         1
Final primal solution.........:  [2. 1.]
Final dual solution.........:  [2.]
Final working set.........:  [1]
Final objective.................: -11
Number of iterations............: 2
Exit: Solution found!


### (H1) Solve a randomly generated QP with $n=10$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [446]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(0)
n = 10
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -3.5741503167386772e-02  5.32e+01  7.28e-04           4         1
     2 -3.5741503167386786e-02  2.67e+01  1.86e-19           5         2
     3 -6.8172713942258267e-01  6.61e+00  7.23e-02           7         3
     4 -1.3495154207496516e+00  5.35e+00  9.65e-02           9         4
     5 -2.8055252351796209e+00  2.93e+00  3.72e-01           2         5
     6 -3.6360548744741310e+00  1.64e+00  6.94e-01          10         6
     7 -3.6913715154445619e+00  3.78e-01  1.00e+00           0         6
Final primal solution.........:  [-1.13012225 -3.19971408  2.28501335  3.30275486 -2.86328241 -2.24287794
  1.2545595   2.93859202 -0.42847311 -0.59571269]
Final dual solution.........:  [0.31100599 0.02203155 0.38342021 0.33305494 0.57922876 0.24769168]
Final working set.........:  [2, 4, 5, 7, 9, 10]
Final objective.................: -3.69

### (H2) Solve a randomly generated QP with $n=10$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [447]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(1)
n = 10
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -8.4456258932183614e-02  2.18e+01  1.77e-03           1         1
     2 -8.4456258932183614e-02  2.29e+01 -0.00e+00           2         2
     3 -8.4456258932183614e-02  2.28e+01 -0.00e+00           3         3
     4 -8.4456258932183614e-02  9.84e+00 -0.00e+00           4         4
     5 -8.4456258932183614e-02  9.84e+00 -0.00e+00           5         5
     6 -8.6355434621287031e-02  2.36e+00  4.72e-04           6         6
     7 -8.6355434621287031e-02  2.02e+00 -6.52e-18           8         7
     8 -8.6355434621287031e-02  8.67e-01 -0.00e+00           9         8
     9 -8.6355434621287031e-02  5.70e-01 -0.00e+00          10         9
  iter                       f   ||p_k||     alpha   new_index   size(W)
    10 -1.1305217086243622e+00  6.14e-01  1.00e+00           0         9
    11 -1.1305217086243622e+00  0.00e+00  1.00e+00 

### (H3) Solve a randomly generated QP with $n=10$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [448]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(2)
n = 10
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -2.4755718358185747e-01  1.27e+01  1.40e-02           1         1
     2 -2.4755718358185747e-01  1.24e+01 -0.00e+00           5         2
     3 -2.4755718358185747e-01  1.35e+01 -0.00e+00           2         3
     4 -2.4755718358185747e-01  1.35e+01 -0.00e+00           4         4
     5 -2.4755718358185747e-01  1.26e+01 -0.00e+00           6         5
     6 -2.4755718358185747e-01  3.48e+00 -0.00e+00           8         6
     7 -2.1323777505554724e+00  3.66e+00  1.00e+00           0         6
     8 -2.1323777505554724e+00  2.22e-16  1.00e+00          -1         5
     9 -2.9693902663939156e+00  2.24e+00  1.00e+00           0         5
  iter                       f   ||p_k||     alpha   new_index   size(W)
    10 -2.9693902663939156e+00  1.94e-16  1.00e+00          -6         4
    11 -2.9751156051076402e+00  1.11e-01  1.00e+00 

### (I1) Solve a randomly generated QP with $n=100$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [449]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(0)
n = 100
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -9.4208840079870834e-02  1.36e+02  4.48e-04           1         1
     2 -9.4208840079870598e-02  3.70e+01 -4.30e-19           2         2
     3 -9.4208840079870682e-02  5.02e+00 -1.11e-18           6         3
     4 -6.6256855400032966e-01  5.62e+00  2.48e-02           7         4
     5 -3.8569953065904241e+00  4.74e+00  1.66e-01           4         5
     6 -1.1177810184895907e+01  3.42e+00  1.00e+00           0         5
Final primal solution.........:  [ 2.5349465   1.47616202 -0.81233401 -1.9673947   0.50797472  3.9452478
  0.850463   -1.34639937  0.78403246  0.6071387 ]
Final dual solution.........:  [0.14745579 0.66572106 0.02092597 0.08580563 0.08642964]
Final working set.........:  [1, 2, 4, 6, 7]
Final objective.................: -11.1778
Number of iterations............: 6
Exit: Solution found!


### (I2) Solve a randomly generated QP with $n=100$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [450]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(1)
n = 100
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -7.2386271329018814e-02  1.78e+02  6.98e-05           2         1
     2 -1.7639600360001281e-01  3.45e+01  5.60e-04           5         2
     3 -9.8481999453380975e-01  1.79e+01  8.54e-03           1         3
     4 -1.2392049526956901e+00  1.60e+01  2.75e-03           6         4
     5 -2.8590332639969898e+00  1.04e+01  2.25e-02           7         5
     6 -4.2944787603331305e+00  8.85e+00  2.33e-02           9         6
     7 -3.3889597503950839e+01  8.70e+00  1.00e+00           0         6
Final primal solution.........:  [ 2.16332601 -4.24445192  1.08380807  2.62199673  0.06146804 -3.43698031
  1.33258886  6.3449915  -4.91032973 -6.07103326]
Final dual solution.........:  [0.0332171  0.21903666 0.65756059 0.41218555 0.39469623 0.03928975]
Final working set.........:  [1, 2, 5, 6, 7, 9]
Final objective.................: -33.88

### (I3) Solve a randomly generated QP with $n=100$ variables and $m=10$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [451]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(2)
n = 100
m = 10
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -8.0705813880190880e-01  2.31e+02  1.19e-03           4         1
     2 -9.5432656453890397e-01  1.93e+02  2.27e-04           2         2
     3 -9.5432656453890397e-01  9.54e+01 -0.00e+00           5         3
     4 -5.1671856997081917e+01  1.97e+01  6.71e-01           6         4
     5 -5.7929460260233775e+01  6.45e+00  1.00e+00           0         4
Final primal solution.........:  [-19.57580221   0.25691291  -2.64124619   2.07079439  -3.63225327
 -13.19838183   3.4956248   -5.38741759  -2.39023977   1.14548659]
Final dual solution.........:  [0.4295956  0.80162363 0.5047895  0.00875238]
Final working set.........:  [2, 4, 5, 6]
Final objective.................: -57.9295
Number of iterations............: 5
Exit: Solution found!


### (J1) Solve a randomly generated QP with $n=10$ variables and $m=100$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [452]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(0)
n = 10
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)
print("Size of working set at time of termination: ", stats['W_size'])

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -1.7307911112722368e-02  5.32e+01  3.53e-04           4         1
     2 -1.7307911112722368e-02  2.66e+01 -0.00e+00           5         2
     3 -1.7307911112722368e-02  7.03e+00 -0.00e+00           7         3
     4 -1.7307911112722368e-02  6.24e+00 -0.00e+00           9         4
     5 -1.7307911112722368e-02  4.09e+00 -0.00e+00           2         5
     6 -1.7307911112722368e-02  4.06e+00 -0.00e+00          10         6
     7 -1.7307911112722368e-02  4.15e+00 -0.00e+00          13         7
     8 -1.7307911112722368e-02  4.24e+00 -0.00e+00          15         8
     9 -1.7307911112722368e-02  4.19e+00 -0.00e+00          16         9
  iter                       f   ||p_k||     alpha   new_index   size(W)
    10 -1.7307911112722368e-02  2.11e+00 -0.00e+00          18        10
    11 -1.7307911112722368e-02  4.08e+00 -0.00e+00 

### (J2) Solve a randomly generated QP with $n=10$ variables and $m=100$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [453]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(1)
n = 10
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)
print("Size of working set at time of termination: ", stats['W_size'])

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -2.3140914879655892e-02  2.18e+01  4.83e-04           1         1
     2 -2.3140914879655885e-02  2.30e+01 -1.36e-19           2         2
     3 -2.3140914879655885e-02  2.30e+01 -0.00e+00           3         3
     4 -2.3140914879655885e-02  9.80e+00 -0.00e+00           4         4
     5 -2.3140914879655885e-02  9.81e+00 -0.00e+00           5         5
     6 -2.3140914879655885e-02  2.65e+00 -0.00e+00           6         6
     7 -2.3140914879655885e-02  1.82e+00 -0.00e+00           8         7
     8 -3.9045369955202129e-02  6.65e-01  2.22e-02          10         8
     9 -9.5701033304873340e-02  7.44e-01  8.46e-02           7         9
  iter                       f   ||p_k||     alpha   new_index   size(W)
    10 -9.5701033304873340e-02  6.65e-01 -0.00e+00          11        10
    11 -9.5701033304873340e-02  6.27e-01 -0.00e+00 

LinAlgError: Singular matrix

### (J3) Solve a randomly generated QP with $n=10$ variables and $m=100$ inequality constraints given $x_0 = \vec{0}$ and $\mathcal{W}_0 = \emptyset$

In [454]:
options = InitOptions()  # Import options

# First, randomly generate the objective function and constraints
np.random.seed(2)
n = 10
m = 100
L = np.random.rand(n,n)
G = np.matmul(L,L.T)
c = 2*(0.5-np.random.rand(n))
A = 2*(0.5-np.random.rand(m,n))
b = -np.random.rand(m)

# Initial iterate and working set
x_0 = np.zeros(n)
W_0 = [False for i in range(m)]

x_sol, lam_sol, W_sol, status, stats = solveIQP_ActiveSet(x_0, W_0, G, c, A, b, options)
print("Size of working set at time of termination: ", stats['W_size'])

  iter                       f   ||p_k||     alpha   new_index   size(W)
     0  0.0000000000000000e+00  0.00e+00  0.00e+00           0         0
     1 -2.4993057386008159e-02  1.27e+01  1.41e-03           1         1
     2 -2.4993057386008159e-02  1.26e+01 -0.00e+00           5         2
     3 -2.4993057386008159e-02  1.36e+01 -0.00e+00           2         3
     4 -2.4993057386008159e-02  1.37e+01 -0.00e+00           4         4
     5 -2.4993057386008159e-02  1.25e+01 -0.00e+00           6         5
     6 -2.4993057386008159e-02  2.52e+00 -0.00e+00           8         6
     7 -2.4993057386008159e-02  2.81e+00 -0.00e+00          15         7
     8 -2.4993057386008159e-02  8.23e-01 -0.00e+00          12         8
     9 -2.4993057386008159e-02  9.07e-01 -0.00e+00          14         9
  iter                       f   ||p_k||     alpha   new_index   size(W)
    10 -2.4993057386008159e-02  1.01e+00 -0.00e+00          18        10
    11 -1.6197298193156243e-03  2.18e+00  1.87e-02 